In [41]:
%%capture
!pip install openai
!pip install tiktoken
!pip install pypdf
!pip install chromadb
!pip install langchain
!pip install unstructured
!pip install pdf2pptx

In [53]:
import tiktoken

In [54]:
model = 'text-davinci-003'
tokenizer = tiktoken.encoding_for_model(model)

In [57]:
# load powerpoint
from langchain.document_loaders import UnstructuredPowerPointLoader

loader = UnstructuredPowerPointLoader("/content/AO.pptx")
data = loader.load()


In [ ]:
ppt_extract = data[0].page_content
print(ppt_extract)

In [58]:
tokens = tokenizer.encode(data[0].page_content)
n_tokens= len(tokens)
total_cost = (n_tokens * 0.02 / 1000)

print("powerpoint loader")
print("Total cost: {}".format(total_cost))


powerpoint loader
Total cost: 0.1648


In [67]:
# load pdf
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/AO.pdf")
pages = loader.load_and_split()

In [68]:
n_tokens = []
for page in pages:
  tokens = tokenizer.encode(page.page_content)
  n_tokens.append(len(tokens))

In [70]:
total_tokens = 0
total_cost = 0

for n in n_tokens:
  total_tokens += n
  total_cost += (n * 0.02 / 1000)

print("pdf loader")
print("Total tokens: {}".format(total_tokens)) 
print("Total cost: {}".format(total_cost)) 

pdf loader
Total tokens: 21507
Total cost: 0.43014000000000013


In [50]:
from langchain.text_splitter import CharacterTextSplitter

In [72]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(pages)

In [77]:
texts[23].page_content

'23 Confidentiel\nExemples de livrables pour le lot 0LOT 0 LOT 1 LOT 2 LOT 3\nExemples de livrables Office 365\nFormation Teams pour les équipes Pégase\nAnalyse d’impacts du projet Parcours Ramsay'

In [78]:
from langchain.vectorstores import Chroma

In [83]:
import os
import getpass
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2022-12-01"
os.environ["OPENAI_API_BASE"] = "https://ai-rd-dev.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = getpass.getpass()


In [84]:
from langchain.llms import AzureOpenAI
from langchain.embeddings import OpenAIEmbeddings
llm = AzureOpenAI(
    deployment_name="davinci003", temperature = 0 
)
embeddings = OpenAIEmbeddings(chunk_size=1)

In [85]:
db = Chroma.from_documents(texts, embeddings)

In [86]:
retriever = db.as_retriever(search_type='similarity')

In [87]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm, chain_type='map_reduce', retriever=retriever)

In [112]:
query = "Combien de profils sont proposées dans la proposition financière ?"

In [113]:
result = qa({"query":query})

In [114]:
result

{'query': 'Combien de profils sont proposées dans la proposition financière ?',
 'result': ' Aucun.'}

In [ ]:
def chat_with_pdf():
  while True:
    query = input('Question: ')
    if query == 'STOP':
      break
    result = qa({'query':query})
    print(result['result'])

In [ ]:
chat_with_pdf()

Question: de quoi parle le rapport ?
 Ce rapport traite des avantages et des questions soulevés par l'intelligence artificielle, notamment l'emploi, la confidentialité des données, la vie privée, la violation des valeurs éthiques et la confiance dans les résultats. Il aborde également le potentiel de l'IA pour améliorer les systèmes de santé et de soins.
Question: comment l'IA peut-elle améliorer les systèmes de santé
 L'IA peut améliorer les systèmes de santé en fournissant des outils pour le traitement et la modélisation de données et de processus complexes, en évaluant d'énormes quantités de publications de recherche, en repérant les corrélations improbables et faibles dans d'énormes ensembles de données, en analysant les images et autres données produites par les systèmes de santé et en développant de nouvelles technologies.


KeyboardInterrupt: ignored